In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import numpy as np

import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

In [2]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

import torch.utils.data
import math


class LayerNorm(nn.Module):
    """
    layer normalization
    Simple layer norm object optionally used with the convolutional encoder.
    """

    def __init__(self, feature_dim, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones((feature_dim,)))
        self.register_parameter("gamma", self.gamma)
        self.beta = nn.Parameter(torch.zeros((feature_dim,)))
        self.register_parameter("beta", self.beta)
        self.eps = eps

    def forward(self, x):
        # x: [batch_size, embed_dim]
        # normalize for each embedding
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # output shape is the same as x
        # Type not match for self.gamma and self.beta??????????????????????
        # output: [batch_size, embed_dim]
        return self.gamma * (x - mean) / (std + self.eps) + self.beta


def get_activation_function(activation, context_str):
    if activation == "leakyrelu":
        return nn.LeakyReLU(negative_slope=0.2)
    elif activation == "relu":
        return nn.ReLU()
    elif activation == "sigmoid":
        return nn.Sigmoid()
    elif activation == 'tanh':
        return nn.Tanh()
    elif activation == 'gelu':
        return nn.GELU()
    else:
        raise Exception("{} activation not recognized.".format(context_str))


class SingleFeedForwardNN(nn.Module):
    """
        Creates a single layer fully connected feed forward neural network.
        this will use non-linearity, layer normalization, dropout
        this is for the hidden layer, not the last layer of the feed forard NN
    """

    def __init__(self, input_dim,
                 output_dim,
                 dropout_rate=None,
                 activation="sigmoid",
                 use_layernormalize=False,
                 skip_connection=False,
                 context_str=''):
        '''

        Args:
            input_dim (int32): the input embedding dim
            output_dim (int32): dimension of the output of the network.
            dropout_rate (scalar tensor or float): Dropout keep prob.
            activation (string): tanh or relu or leakyrelu or sigmoid
            use_layernormalize (bool): do layer normalization or not
            skip_connection (bool): do skip connection or not
            context_str (string): indicate which spatial relation encoder is using the current FFN

        '''
        super(SingleFeedForwardNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        if dropout_rate is not None:
            self.dropout = nn.Dropout(p=dropout_rate)
        else:
            self.dropout = None

        self.act = get_activation_function(activation, context_str)

        if use_layernormalize:
            # the layer normalization is only used in the hidden layer, not the last layer
            self.layernorm = nn.LayerNorm(self.output_dim)
        else:
            self.layernorm = None

        # the skip connection is only possible, if the input and out dimention is the same
        if self.input_dim == self.output_dim:
            self.skip_connection = skip_connection
        else:
            self.skip_connection = False

        self.linear = nn.Linear(self.input_dim, self.output_dim)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, input_tensor):
        '''
        Args:
            input_tensor: shape [batch_size, ..., input_dim]
        Returns:
            tensor of shape [batch_size,..., output_dim]
            note there is no non-linearity applied to the output.

        Raises:
            Exception: If given activation or normalizer not supported.
        '''
        assert input_tensor.size()[-1] == self.input_dim
        # Linear layer
        output = self.linear(input_tensor)
        # non-linearity
        output = self.act(output)
        # dropout
        if self.dropout is not None:
            output = self.dropout(output)

        # skip connection
        if self.skip_connection:
            output = output + input_tensor

        # layer normalization
        if self.layernorm is not None:
            output = self.layernorm(output)

        return output


class MultiLayerFeedForwardNN(nn.Module):
    """
        Creates a fully connected feed forward neural network.
        N fully connected feed forward NN, each hidden layer will use non-linearity, layer normalization, dropout
        The last layer do not have any of these
    """

    def __init__(self, input_dim,
                 output_dim,
                 num_hidden_layers=0,
                 dropout_rate=None,
                 hidden_dim=-1,
                 activation="sigmoid",
                 use_layernormalize=False,
                 skip_connection=False,
                 context_str=None):
        '''

        Args:
            input_dim (int32): the input embedding dim
            num_hidden_layers (int32): number of hidden layers in the network, set to 0 for a linear network.
            output_dim (int32): dimension of the output of the network.
            dropout (scalar tensor or float): Dropout keep prob.
            hidden_dim (int32): size of the hidden layers
            activation (string): tanh or relu
            use_layernormalize (bool): do layer normalization or not
            context_str (string): indicate which spatial relation encoder is using the current FFN

        '''
        super(MultiLayerFeedForwardNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_hidden_layers = num_hidden_layers
        self.dropout_rate = dropout_rate
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.use_layernormalize = use_layernormalize
        self.skip_connection = skip_connection
        self.context_str = context_str

        self.layers = nn.ModuleList()
        if self.num_hidden_layers <= 0:
            self.layers.append(SingleFeedForwardNN(input_dim=self.input_dim,
                                                   output_dim=self.output_dim,
                                                   dropout_rate=self.dropout_rate,
                                                   activation=self.activation,
                                                   use_layernormalize=False,
                                                   skip_connection=False,
                                                   context_str=self.context_str))
        else:
            self.layers.append(SingleFeedForwardNN(input_dim=self.input_dim,
                                                   output_dim=self.hidden_dim,
                                                   dropout_rate=self.dropout_rate,
                                                   activation=self.activation,
                                                   use_layernormalize=self.use_layernormalize,
                                                   skip_connection=self.skip_connection,
                                                   context_str=self.context_str))

            for i in range(self.num_hidden_layers - 1):
                self.layers.append(SingleFeedForwardNN(input_dim=self.hidden_dim,
                                                       output_dim=self.hidden_dim,
                                                       dropout_rate=self.dropout_rate,
                                                       activation=self.activation,
                                                       use_layernormalize=self.use_layernormalize,
                                                       skip_connection=self.skip_connection,
                                                       context_str=self.context_str))

            self.layers.append(SingleFeedForwardNN(input_dim=self.hidden_dim,
                                                   output_dim=self.output_dim,
                                                   dropout_rate=self.dropout_rate,
                                                   activation=self.activation,
                                                   use_layernormalize=False,
                                                   skip_connection=False,
                                                   context_str=self.context_str))

    def forward(self, input_tensor):
        '''
        Args:
            input_tensor: shape [batch_size, ..., input_dim]
        Returns:
            tensor of shape [batch_size, ..., output_dim]
            note there is no non-linearity applied to the output.

        Raises:
            Exception: If given activation or normalizer not supported.
        '''
        assert input_tensor.size()[-1] == self.input_dim
        output = input_tensor
        for i in range(len(self.layers)):
            output = self.layers[i](output)

        return output

In [3]:
import math

def _cal_freq_list(freq_init, frequency_num, max_radius, min_radius):
    if freq_init == "random":
        # the frequence we use for each block, alpha in ICLR paper
        # freq_list shape: (frequency_num)
        freq_list = np.random.random(size=[frequency_num]) * max_radius
    elif freq_init == "geometric":
        # freq_list = []
        # for cur_freq in range(frequency_num):
        #     base = 1.0/(np.power(max_radius, cur_freq*1.0/(frequency_num-1)))
        #     freq_list.append(base)

        # freq_list = np.asarray(freq_list)

        log_timescale_increment = math.log(float(max_radius) / float(min_radius)) / (
            frequency_num * 1.0 - 1
        )

        timescales = min_radius * np.exp(
            np.arange(frequency_num).astype(float) * log_timescale_increment
        )

        freq_list = 1.0 / timescales
    elif freq_init == "nerf":
        """
        compute according to NeRF position encoding,
        Equation 4 in https://arxiv.org/pdf/2003.08934.pdf
        2^{0}*pi, ..., 2^{L-1}*pi
        """
        #
        freq_list = np.pi * np.exp2(np.arange(frequency_num).astype(float))

    return freq_list

class PositionEncoder(nn.Module):
  def __init__(self, coord_dim=2, device="cuda"):
      super(PositionEncoder, self).__init__()
      self.coord_dim = coord_dim
      self.device = device
      self.pos_enc_output_dim = None  # self.cal_pos_enc_output_dim()

  def cal_pos_enc_output_dim(self):
      raise NotImplementedError(
          "The 'pos_enc_output_dim' property should be implemented by subclasses."
      )

  def forward(self, coords):
      raise NotImplementedError(
          "This method should be implemented by subclasses.")


class LocationEncoder(nn.Module):
  def __init__(self, spa_embed_dim, coord_dim=2, device="cuda"):
      super(LocationEncoder, self).__init__()
      self.spa_embed_dim = spa_embed_dim
      self.coord_dim = coord_dim
      self.device = device

  def vil_attribute(self):
      raise NotImplementedError("Subclasses must implement this property.")

  def forward(self, coords):
      raise NotImplementedError(
          "This method should be implemented by subclasses.")

class GridCellSpatialRelationPositionEncoder(PositionEncoder):
    """
    Given a list of(deltaX, deltaY), encode them using the position encoding function
    """

    def __init__(
        self,
        coord_dim=2,
        frequency_num=16,
        max_radius=10000,
        min_radius=10,
        freq_init="geometric",
        device="cuda",
    ):
        """
        Args:
            coord_dim: the dimention of space, 2D, 3D, or other
            frequency_num: the number of different sinusoidal with different frequencies / wavelengths
            max_radius: the largest context radius this model can handle
        """
        super().__init__(coord_dim=coord_dim, device=device)
        self.frequency_num = frequency_num
        self.max_radius = max_radius
        self.min_radius = min_radius
        self.freq_init = freq_init
        # the frequence we use for each block, alpha in ICLR paper
        self.cal_freq_list()
        self.cal_freq_mat()

        self.pos_enc_output_dim = self.cal_pos_enc_output_dim()

    def cal_elementwise_angle(self, coord, cur_freq):
        """
        Args:
            coord: the deltaX or deltaY
            cur_freq: the frequency
        """
        return coord / (
            np.power(self.max_radius, cur_freq *
                     1.0 / (self.frequency_num - 1))
        )

    def cal_coord_embed(self, coords_tuple):
        embed = []
        for coord in coords_tuple:
            for cur_freq in range(self.frequency_num):
                embed.append(
                    math.sin(self.cal_elementwise_angle(coord, cur_freq)))
                embed.append(
                    math.cos(self.cal_elementwise_angle(coord, cur_freq)))
        # embed: shape (pos_enc_output_dim)
        return embed

    def cal_pos_enc_output_dim(self):
        # compute the dimention of the encoded spatial relation embedding
        return int(self.coord_dim * self.frequency_num * 2)

    def cal_freq_list(self):
        self.freq_list = _cal_freq_list(
            self.freq_init, self.frequency_num, self.max_radius, self.min_radius
        )

    def cal_freq_mat(self):
        # freq_mat shape: (frequency_num, 1)
        freq_mat = np.expand_dims(self.freq_list, axis=1)
        # self.freq_mat shape: (frequency_num, 2)
        self.freq_mat = np.repeat(freq_mat, 2, axis=1)

    def make_output_embeds(self, coords):
        if type(coords) == np.ndarray:
            assert self.coord_dim == np.shape(coords)[2]
            coords = list(coords)
        elif type(coords) == list:  # noqa: E721
            assert self.coord_dim == len(coords[0][0])
        else:
            raise Exception(
                "Unknown coords data type for GridCellSpatialRelationEncoder"
            )

        # coords_mat: shape (batch_size, num_context_pt, 2)
        coords_mat = np.asarray(coords).astype(float)
        batch_size = coords_mat.shape[0]
        num_context_pt = coords_mat.shape[1]
        # coords_mat: shape (batch_size, num_context_pt, 2, 1)
        coords_mat = np.expand_dims(coords_mat, axis=3)
        # coords_mat: shape (batch_size, num_context_pt, 2, 1, 1)
        coords_mat = np.expand_dims(coords_mat, axis=4)
        # coords_mat: shape (batch_size, num_context_pt, 2, frequency_num, 1)
        coords_mat = np.repeat(coords_mat, self.frequency_num, axis=3)
        # coords_mat: shape (batch_size, num_context_pt, 2, frequency_num, 2)
        coords_mat = np.repeat(coords_mat, 2, axis=4)
        # spr_embeds: shape (batch_size, num_context_pt, 2, frequency_num, 2)
        spr_embeds = coords_mat * self.freq_mat

        # make sinuniod function
        # sin for 2i, cos for 2i+1
        # spr_embeds: (batch_size, num_context_pt, 2*frequency_num*2=pos_enc_output_dim)
        spr_embeds[:, :, :, :, 0::2] = np.sin(
            spr_embeds[:, :, :, :, 0::2])  # dim 2i
        spr_embeds[:, :, :, :, 1::2] = np.cos(
            spr_embeds[:, :, :, :, 1::2])  # dim 2i+1

        # (batch_size, num_context_pt, 2*frequency_num*2)
        spr_embeds = np.reshape(spr_embeds, (batch_size, num_context_pt, -1))

        return spr_embeds

    def forward(self, coords):
        """
        Given a list of coords(deltaX, deltaY), give their spatial relation embedding
        Args:
            coords: a python list with shape(batch_size, num_context_pt, coord_dim)
        Return:
            sprenc: Tensor shape(batch_size, num_context_pt, position_embed_dim)
        """

        spr_embeds = self.make_output_embeds(coords)
        spr_embeds = torch.FloatTensor(spr_embeds).to(self.device)

        # sprenc: shape (batch_size, num_context_pt, spa_embed_dim)

        return spr_embeds


class GridCellSpatialRelationLocationEncoder(LocationEncoder):
    def __init__(
        self,
        spa_embed_dim=64,
        coord_dim=2,
        frequency_num=16,
        max_radius=10000,
        min_radius=10,
        freq_init="geometric",
        device="cuda",
        ffn_act="relu",
        ffn_num_hidden_layers=1,
        ffn_dropout_rate=0.5,
        ffn_hidden_dim=256,
        ffn_use_layernormalize=True,
        ffn_skip_connection=True,
        ffn_context_str="GridCellSpatialRelationEncoder",
    ):
        super().__init__(spa_embed_dim, coord_dim, device)
        self.frequency_num = frequency_num
        self.max_radius = max_radius
        self.min_radius = min_radius
        self.freq_init = freq_init
        self.ffn_act = ffn_act
        self.ffn_num_hidden_layers = ffn_num_hidden_layers
        self.ffn_dropout_rate = ffn_dropout_rate
        self.ffn_hidden_dim = ffn_hidden_dim
        self.ffn_use_layernormalize = ffn_use_layernormalize
        self.ffn_skip_connection = ffn_skip_connection

        self.position_encoder = GridCellSpatialRelationPositionEncoder(
            coord_dim=coord_dim,
            frequency_num=frequency_num,
            max_radius=max_radius,
            min_radius=min_radius,
            freq_init=freq_init,
            device=device,
        )
        self.ffn = MultiLayerFeedForwardNN(
            input_dim=self.position_encoder.pos_enc_output_dim,
            # input_dim=int(4 * frequency_num),
            output_dim=self.spa_embed_dim,
            num_hidden_layers=self.ffn_num_hidden_layers,
            dropout_rate=ffn_dropout_rate,
            hidden_dim=self.ffn_hidden_dim,
            activation=self.ffn_act,
            use_layernormalize=self.ffn_use_layernormalize,
            skip_connection=ffn_skip_connection,
            context_str=ffn_context_str,
        )

    def forward(self, coords):
        spr_embeds = self.position_encoder(coords)
        sprenc = self.ffn(spr_embeds)

        return sprenc

In [4]:
class LocationContrastiveModel(nn.Module):
    def __init__(self, embed_dim=64):
        super().__init__()
        self.encoder = GridCellSpatialRelationLocationEncoder(spa_embed_dim=64,
                                                 coord_dim=2,
                                                 frequency_num=16,
                                                 max_radius=1,
                                                 min_radius=0.0001,
                                                 ffn_act='leakyrelu',
                                                 freq_init='geometric',
                                                 ffn_num_hidden_layers=1,
                                                 ffn_dropout_rate=0.5,
                                                 ffn_use_layernormalize=True,
                                                 ffn_skip_connection=True,
                                                 ffn_hidden_dim=512,
                                                 device='cpu')
        self.projector = nn.Linear(64, embed_dim)

    def forward(self, coords):
        #coords_numpy = coords.detach().numpy()
        #coords_numpy = np.expand_dims(coords_numpy, axis=1)
        #z = torch.squeeze(self.encoder(coords_numpy))
        #z = self.projector(z)
        #return F.normalize(z, dim=-1)

        coords = coords.unsqueeze(1).numpy()
        z = self.encoder(coords)
        z = z.squeeze(1)
        z = self.projector(z)
        return F.normalize(z, dim=-1)

    def contrastive_loss(self, z_i, z_j, label, temperature=0.1):
      sim = F.cosine_similarity(z_i, z_j)
      pos = label * torch.exp(sim / temperature)
      neg = (1 - label) * torch.exp(sim / temperature)
      loss = -torch.log((pos + 1e-8) / (pos + neg + 1e-8))
      return loss.mean()

In [5]:
import numpy as np
from sklearn.neighbors import BallTree

EARTH_RADIUS_KM = 6371.0

def latlon_to_radians(coords):
    return np.radians(coords)

def spatial_proximity_pairs_kdtree(coords, r_pos_km=1.0, r_neg_km=5.0, max_pairs=50000):
    coords_rad = latlon_to_radians(coords)

    tree = BallTree(coords_rad, metric="haversine")

    r_pos_rad = r_pos_km / EARTH_RADIUS_KM
    r_neg_rad = r_neg_km / EARTH_RADIUS_KM

    pairs = []

    for i, coord in enumerate(coords_rad):
        idx_pos = tree.query_radius(coord.reshape(1, -1), r=r_pos_rad)[0]
        idx_pos = idx_pos[idx_pos != i]
        for j in idx_pos:
            pairs.append((i, j, 1))

        idx_close = tree.query_radius(coord.reshape(1, -1), r=r_neg_rad)[0]
        all_idx = np.arange(len(coords))
        idx_neg = np.setdiff1d(all_idx, np.append(idx_close, i))

        if len(idx_neg) > 5:
            idx_neg = np.random.choice(idx_neg, 5, replace=False)
        for j in idx_neg:
            pairs.append((i, j, 0))

    if len(pairs) > max_pairs:
        pairs = pairs[:max_pairs]

    return pairs

import numpy as np
from sklearn.neighbors import BallTree


def latlon_to_radians(coords):
    return np.radians(coords.astype(np.float32))

def spatial_proximity_pairs_kdtree_fast(
    coords, r_pos_km=1.0, r_neg_km=20.0, max_pairs=50_000,
    k_neg_per_i=5, max_pos_per_i=None, seed=42, leaf_size=40
):

    rng = np.random.default_rng(seed)
    coords = np.asarray(coords, dtype=np.float32)
    N = coords.shape[0]
    coords_rad = latlon_to_radians(coords)
    r_pos_rad = r_pos_km / EARTH_RADIUS_KM
    r_neg_rad = r_neg_km / EARTH_RADIUS_KM

    tree = BallTree(coords_rad, metric="haversine", leaf_size=leaf_size)

    pos_lists = tree.query_radius(coords_rad, r=r_pos_rad, return_distance=False)
    close_lists = tree.query_radius(coords_rad, r=r_neg_rad, return_distance=False)  # "próximos" (não-negativos)

    pairs_i = []
    pairs_j = []
    pairs_y = []

    def add_pair(i, j, y):
        pairs_i.append(i); pairs_j.append(j); pairs_y.append(y)

    total = 0
    for i in range(N):
        pos_i = pos_lists[i]
        pos_i = pos_i[pos_i != i]
        if pos_i.size:
            if max_pos_per_i is not None and pos_i.size > max_pos_per_i:
                pos_i = rng.choice(pos_i, size=max_pos_per_i, replace=False)
            for j in pos_i:
                add_pair(i, int(j), 1)
                total += 1
                if total >= max_pairs:
                    return list(zip(pairs_i, pairs_j, pairs_y))

        close_set = set(map(int, close_lists[i]))
        close_set.add(i)

        got = 0
        trials = 0
        max_trials = 50 * k_neg_per_i
        while got < k_neg_per_i and trials < max_trials:
            j = int(rng.integers(0, N))
            trials += 1
            if j in close_set:
                continue
            add_pair(i, j, 0)
            total += 1
            got += 1
            if total >= max_pairs:
                return list(zip(pairs_i, pairs_j, pairs_y))

    return list(zip(pairs_i, pairs_j, pairs_y))


In [6]:
from torch.utils.data import Dataset

class SpatialContrastiveDataset(Dataset):
    def __init__(self, coords, r_pos_km=1.0, r_neg_km=1.0, max_pairs=50000):
        self.coords = np.array(coords)
        self.pairs = spatial_proximity_pairs_kdtree_fast(
            self.coords, r_pos_km, r_neg_km, max_pairs,
            k_neg_per_i=5, max_pos_per_i=100000, seed=42
        )

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i, j, label = self.pairs[idx]
        coord_i = self.coords[i]
        coord_j = self.coords[j]
        return coord_i, coord_j, label

In [7]:
import pandas as pd

checkins = pd.read_csv('/content/checkins_North Carolina.csv')[['latitude', 'longitude', 'category','placeid']]
coords = checkins[['latitude', 'longitude']].values
categories = checkins[['category']].values
placeid = checkins[['placeid']].values

In [8]:
print("len(df):", len(checkins))
print("coords:", np.asarray(coords).shape)
print("categories:", np.asarray(categories).shape)
print("placeid:", np.asarray(placeid).shape)


len(df): 229247
coords: (229247, 2)
categories: (229247, 1)
placeid: (229247, 1)


In [9]:
from torch.utils.data import DataLoader

dataset = SpatialContrastiveDataset(coords, r_pos_km=1.0, r_neg_km=20)
loader = DataLoader(dataset, batch_size=64)

device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model = LocationContrastiveModel(embed_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    total_loss = 0
    for coord_i, coord_j, label in loader:
        coord_i, coord_j, label = coord_i.to(device), coord_j.to(device), label.to(device)

        z_i = model(coord_i)
        z_j = model(coord_j)

        loss = model.contrastive_loss(z_i, z_j, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss/len(loader):.4f}")

Epoch 1: Loss = 0.0100
Epoch 2: Loss = 0.0063
Epoch 3: Loss = 0.0055
Epoch 4: Loss = 0.0053
Epoch 5: Loss = 0.0052
Epoch 6: Loss = 0.0052
Epoch 7: Loss = 0.0051
Epoch 8: Loss = 0.0051
Epoch 9: Loss = 0.0051
Epoch 10: Loss = 0.0051
Epoch 11: Loss = 0.0051
Epoch 12: Loss = 0.0051
Epoch 13: Loss = 0.0051
Epoch 14: Loss = 0.0051
Epoch 15: Loss = 0.0051
Epoch 16: Loss = 0.0051
Epoch 17: Loss = 0.0051
Epoch 18: Loss = 0.0051
Epoch 19: Loss = 0.0051
Epoch 20: Loss = 0.0051
Epoch 21: Loss = 0.0051
Epoch 22: Loss = 0.0051
Epoch 23: Loss = 0.0051
Epoch 24: Loss = 0.0051
Epoch 25: Loss = 0.0051
Epoch 26: Loss = 0.0051
Epoch 27: Loss = 0.0051
Epoch 28: Loss = 0.0051
Epoch 29: Loss = 0.0051
Epoch 30: Loss = 0.0051
Epoch 31: Loss = 0.0050
Epoch 32: Loss = 0.0051
Epoch 33: Loss = 0.0050
Epoch 34: Loss = 0.0050
Epoch 35: Loss = 0.0051
Epoch 36: Loss = 0.0051
Epoch 37: Loss = 0.0050
Epoch 38: Loss = 0.0051
Epoch 39: Loss = 0.0051
Epoch 40: Loss = 0.0051
Epoch 41: Loss = 0.0051
Epoch 42: Loss = 0.0050
E

In [10]:
loc_embeds = model(torch.Tensor(coords))

# loc_embed: shape [batch_size, spa_embed_dim]
loc_embeds

tensor([[ 0.1359, -0.1288, -0.0153,  ..., -0.1930,  0.1096,  0.0022],
        [ 0.1382, -0.1270, -0.0140,  ..., -0.1954,  0.1072,  0.0042],
        [ 0.1377, -0.1269, -0.0135,  ..., -0.1954,  0.1092,  0.0030],
        ...,
        [ 0.1366, -0.1280, -0.0153,  ..., -0.1932,  0.1096,  0.0019],
        [ 0.1360, -0.1290, -0.0157,  ..., -0.1938,  0.1093,  0.0041],
        [-0.1370,  0.1281,  0.0152,  ...,  0.1941, -0.1091, -0.0025]],
       grad_fn=<DivBackward0>)

In [11]:
loc_embeds = loc_embeds.detach().numpy()

In [12]:
placeid.shape

(229247, 1)

In [13]:
loc_embeds.shape

(229247, 64)

In [14]:
categories.shape

(229247, 1)

In [17]:
n_embeds = loc_embeds.shape[1]
embed_cols = [f"{i}" for i in range(n_embeds)]

location_encoder_embeddings = np.hstack([placeid, loc_embeds, categories])

cols = ["placeid"] + embed_cols + ["category"]
df_location_embeddings = pd.DataFrame(location_encoder_embeddings, columns=cols)

df_location_embeddings.to_csv("/content/drive/MyDrive/region-embedding-benchmark-main/region-embedding-benchmark-main/location_encoder_embeddings.csv", index=False)

df_location_embeddings.head()


,placeid,0,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,category
0,18643,0.135915,-0.128779,-0.015307,0.078107,0.144131,-0.008117,-0.087305,-0.118264,-0.038017,...,0.122323,-0.171165,0.174069,0.036517,0.04361,-0.137194,-0.193003,0.109563,0.002151,Travel
1,18643,0.138225,-0.126965,-0.014012,0.074297,0.143001,-0.005076,-0.090016,-0.116325,-0.036208,...,0.124488,-0.173152,0.17214,0.034003,0.042162,-0.134889,-0.195412,0.107227,0.004232,Travel
2,18643,0.137746,-0.126945,-0.013532,0.073313,0.142735,-0.003743,-0.088067,-0.117575,-0.03663,...,0.123601,-0.172605,0.172024,0.035879,0.041053,-0.134924,-0.195426,0.109206,0.00304,Travel
3,18643,0.135977,-0.128141,-0.01571,0.076644,0.142893,-0.008474,-0.08764,-0.118818,-0.037467,...,0.122057,-0.171527,0.173708,0.036208,0.043714,-0.136391,-0.19335,0.108647,0.002667,Travel
4,18643,0.136159,-0.128728,-0.01543,0.076095,0.141372,-0.007071,-0.087071,-0.118826,-0.038967,...,0.125129,-0.170192,0.174487,0.037381,0.043468,-0.137288,-0.192695,0.109605,0.001302,Travel


In [18]:
df = pd.read_csv("/content/drive/MyDrive/region-embedding-benchmark-main/region-embedding-benchmark-main/location_encoder_embeddings.csv")
placeids = df["placeid"].astype(str).tolist()
E = df[[c for c in df.columns if c.isnumeric()]].to_numpy(dtype=np.float32)

torch.save({
    "embeddings": torch.from_numpy(E),
    "placeids": placeids
}, "/content/drive/MyDrive/region-embedding-benchmark-main/region-embedding-benchmark-main/poi_embeddings_location.pt")


In [22]:
location_encoder_embeddings

array([[18643, 0.13591530919075012, -0.12877854704856873, ...,
        0.1095631942152977, 0.002151458989828825, 'Travel'],
       [18643, 0.138225257396698, -0.12696504592895508, ...,
        0.10722718387842178, 0.00423175934702158, 'Travel'],
       [18643, 0.13774560391902924, -0.1269453912973404, ...,
        0.10920625180006027, 0.0030395614448934793, 'Travel'],
       ...,
       [7356240, 0.1365596055984497, -0.12803371250629425, ...,
        0.10960288345813751, 0.0019031449919566512, 'Shopping'],
       [7514329, 0.13597039878368378, -0.12895168364048004, ...,
        0.10928715765476227, 0.004107913468033075, 'Food'],
       [7394730, -0.1369772106409073, 0.12806908786296844, ...,
        -0.10911385715007782, -0.0025015510618686676, 'Food']],
      dtype=object)

In [ ]:
len(location_encoder_embeddings)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

embeds_np = loc_embeds[:5000]

tsne = TSNE(n_components=2, random_state=42)
embeds_2d = tsne.fit_transform(embeds_np)

plt.figure(figsize=(8, 6))
plt.scatter(embeds_2d[:, 0], embeds_2d[:, 1], s=10, alpha=0.7)
plt.title("Location Embeddings (t-SNE 2D)")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True)
plt.show()

KeyboardInterrupt: 